In [1]:
import pandas as pd
import geopandas as gpd
import os
from os import walk
import zipfile
import re
import numpy as np
from itertools import chain

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [108]:
from itertools import chain

In [4]:
inpath = r'L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping'

In [5]:
wdpath = inpath + '\\DSL data originals\Set001'

In [6]:
txpath = inpath + '\\GIS\ORMAP_data\Downloaded zip files'

In [7]:
newpath = os.path.join(inpath, "test")

In [8]:
# create a temporary folder to run a test
os.mkdir(newpath)

In [20]:
def list_files(path, folder=False):
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
        break
    if folder:
        f = [x[0] for x in os.walk(path)]
    return f

In [14]:
list_files(wdpath)

['ClatsopCoDelsDets2017-20220601.xlsx',
 'CoosCo2017-20220601.xlsx',
 'DeschutesCoDelDet2017-20220601.xlsx',
 'HarneyCo2017-20220601.xlsx',
 'JacksonCo2017-20220601.xlsx',
 'JosephineCo2017-20220601.xlsx',
 'LakeCoDelDet2017-20220601.xlsx',
 'MarionCoDelDet2017-20220601.xlsx',
 '~$ClatsopCoDelsDets2017-20220601.xlsx',
 '~$CoosCo2017-20220601.xlsx']

In [15]:
list_files(txpath)

['2012 Taxlots.zip',
 '2014_state_agency_use_taxlots.gdb.zip',
 '2015_state_agency_use_taxlots.gdb.zip',
 '2016ShareableTaxLotsGDBJuneUpdate.zip',
 'Morrow2015.zip',
 'Shared_Taxlots_2017_GDB.zip',
 'Shared_Taxlots_2018_GDB.zip',
 'Shared_Taxlots_2019_GDB.zip',
 'Shared_Taxlots_2020_GDB.zip',
 'Shared_Taxlots_2021_GDB.zip',
 'Shared_Taxlots_2022_GDB.zip',
 'State Data2009.gdb.zip',
 'taxlots_2011.zip']

In [25]:
year = 2022

In [26]:
with zipfile.ZipFile(os.path.join(txpath, 'Shared_Taxlots_'+str(year)+'_GDB.zip'), 'r') as zip_ref:
    zip_ref.extractall(newpath)

In [27]:
# how the original taxlot data is stored
list_files(newpath, folder=True)[1]

['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test\\GDB',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test\\GDB\\Shared Taxlots.gdb']

In [28]:
# rename the folder
os.renames(list_files(newpath, folder=True)[1], newpath+'\\Taxlots'+str(year))

In [29]:
datafile = os.path.join(wdpath, 'ClatsopCoDelsDets2017-20220601.xlsx')

In [31]:
# check the sheets in the wd spreadsheets
xl = pd.ExcelFile(datafile)
xl.sheet_names

['Delineations-Determinations', 'Reorg']

In [32]:
# read the reorganized table in the raw data
wd_dt = pd.read_excel(datafile, sheet_name='Reorg')

In [33]:
wd_dt.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,Coord-Source,DocumentName,DecisionLink,is_batch_file,status_name,received_date,response_date,reissuance_response_date,project_id,site_id
0,WD2017-0003,06N10W03DA,500 (partial),McCormick Rd .025 mi S of Hillila Rd,Gearhart,Clatsop,McCormick Rd 06N10W03DA,Residential,46.030197,-123.905217,DSL Verified,WD2017-0003final,https://docs.dsl.state.or.us/PublicReview/docv...,N,Approved,2017-01-03,2017-03-09,NaN,NaN,103777
1,WD2017-0033,08N08W25,101,"W of 92260 Koppisch Rd, Astoria",Astoria,Clatsop,08N08W25,site evaluation,46.155778,-123.610751,DSL Verified,WD20170033AgencyDecision,https://docs.dsl.state.or.us/PublicReview/Elec...,Y,Completed by DSL,2017-01-12,2017-01-30,NaN,NaN,104259
2,WD2017-0056,08N10W09B,"2500, ROWs (both partial)",Warrenton Dr and NW 17th St,Warrenton,Clatsop,Gann Dev 08N10W09B,residential development,46.190638,-123.932177,DSL Verified,WD2017-0056final,https://docs.dsl.state.or.us/PublicReview/docv...,N,Approved,2017-02-06,2017-05-17,NaN,NaN,104411


In [34]:
wd_dt.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id'],
      dtype='object')

In [38]:
txfilepath = os.path.join(newpath, 'Taxlots'+str(year), 'Shared Taxlots.gdb')

In [39]:
# this will take a while
tx_dt = gpd.read_file(txfilepath, layer='Taxlots')

In [42]:
# just check a smaller set
tx_dt_s = gpd.read_file(txfilepath, layer='Taxlots', rows=10)

In [44]:
tx_dt_s.columns

Index(['County', 'Town', 'TownPart', 'TownDir', 'Range', 'RangePart',
       'RangeDir', 'SecNumber', 'Qtr', 'QtrQtr', 'Anomaly', 'MapSufType',
       'MapNumber', 'ORMapNum', 'Taxlot', 'MapTaxlot', 'ORTaxlot',
       'TaxlotAcre', 'ReliaCode', 'TaxLotFeet', 'MapClass', 'MapRelCode',
       'MapSufNum', 'REFLink', 'SpecialInt', 'INSTYEAR', 'PRIMACCNUM',
       'OWNERLINE1', 'MAILADD1', 'MAILADD2', 'MAILCITY', 'MAILZIP', 'PRPCLASS',
       'SIMAPTAX', 'OWNERLINE2', 'OWNERLINE3', 'AGENTNAME', 'MAILSTATE',
       'MAILCNTRY', 'SITEADDNAM', 'SITEADDCTY', 'SITEZIP', 'INSTMONTH',
       'INSTID', 'INSTTYPE', 'DWELLING', 'PRPCLSDSC', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')

In [46]:
# check the format of ORTaxlot
tx_dt_s.ORTaxlot.head(3)

0    0106.00S38.00E0000--000000300
1    0109.00S40.00E2800--000000800
2    0106.00S38.00E0000--0000ROADS
Name: ORTaxlot, dtype: object

In [47]:
# create a spreadsheet to create a dictionary
cnt_ID = pd.read_excel(r'T:\DCProjects\EPA-WD\CNT_Code.xlsx')

In [48]:
# create a dictionary to look up county code
cnt_dict = dict(zip(cnt_ID.COUNTY, cnt_ID.ID))

In [51]:
# check how parcel id is noted
wd_dt.parcel_id.unique()

array(['500 (partial)', '101', '2500, ROWs (both partial)',
       '302 (partial)', '2100 (partial)', '6600', '100', '204', '200,400',
       '1300,1400,1401,ROW (all partial)', '4600 (partial)', '701,703',
       '200', '900,1000 (all partial)', '4900,5000 (all partial)',
       '10700', '3502,3800', 'NW 11th St ROW (partial)', '1300 (partial)',
       '701', '3602', '2102, 900 (both partial)', '12308', '3200,3300',
       '4700,4600,4500,4400,4300,4200,4100,ROW,all part',
       '111,SW 2nd street ROW (all partial)',
       '3400,2900,2800,SW 2nd street ROW (all partial)',
       '200,SW 2nd street ROW (all partial)', '300 (partial)', '12400',
       '500', '2100', '600 (partial)',
       '600,700,800,900,1000,1100,1200,1300', '1100,1201', '1700',
       '408, 406, ROW (all partial)', '2310, ROW (both partial)', '2903',
       '8300', '1529', '7600', '1200', '402, 403', '1201', '1300',
       '408 (portion)', '900', '1800', '800', '400', '1204 (partial)',
       '1000', '103,105', 'R

In [69]:
# this id looks different
'100-10600 (106 TLs)'.replace('(','').replace(')','')  

'100-10600 106 TLs'

In [86]:
# function to remove duplicates in the parcel ids
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

In [84]:
# function to get all the lot numbers
def get_lot_numbers(text):
    # remove parenthesis from text
    txt = text.replace('(','').replace(')','')
    # split the text
    lot_list = re.split(",|, | |-", txt)
    # remove text elements
    l = []
    # in case there are still number-letter strings (e.g., '1a')
    for t in [lot for lot in lot_list if ~lot.isnumeric()]:
        if any(c.isdigit() for c in t):
            l.append(re.sub('\D', '', t))
    res = unique([lot for lot in lot_list if lot.isnumeric()] + l)
    return res

In [115]:
# use a random data subset for a test run
tdf = wd_dt.sample(n=3, random_state=1)

In [116]:
# get a list of lot numbers in each parcel id record
tdf.loc[:, 'lots'] = tdf.parcel_id.apply(lambda x: get_lot_numbers(x))

In [117]:
# repeat the rows based on the number of lot numbers
ndf = tdf.reindex(tdf.index.repeat(tdf.lots.str.len()))

In [119]:
# add the column to list the lot for all
ndf.loc[:, 'lot'] = list(chain.from_iterable(tdf.lots.values.tolist()))

In [127]:
# get county code
ndf.loc[:, 'cnt_code'] = ndf.county.map(cnt_dict)

In [159]:
# get OR taxlot IDs for wd data
ndf.loc[:, 'ORTaxlot'] = ndf[['cnt_code', 'trsqq', 'lot']].apply(lambda row: str(row.cnt_code).zfill(2) + row.trsqq[0:2] + '.00' + row.trsqq[2:5] + '.00'+ row.trsqq[5:8] + '{:0<10}'.format(row.trsqq)[8] + '{:0<10}'.format(row.trsqq)[9] + '--' + ('000000000' + row.lot)[-9:], axis = 1) 

In [147]:
ndf.trsqq.unique()

array(['08N10W26', '07N10W16', '08N10W22DA'], dtype=object)

In [160]:
#0106.00S38.00E0000--000000300
ndf.ORTaxlot.unique()

array(['0408.00N10.00W2600--000000500', '0407.00N10.00W1600--000000200',
       '0407.00N10.00W1600--000000300', '0408.00N10.00W22DA--000001001',
       '0408.00N10.00W22DA--000001002', '0408.00N10.00W22DA--000001100'],
      dtype=object)

In [155]:
txlist = list(tx_dt.ORTaxlot.unique())

In [162]:
# check if the taxlot ID is in the correct format
all([x in txlist for x in ndf.ORTaxlot.unique()])

True

In [163]:
# get a taxlot data subset for a join test run
tx_dt_s = tx_dt[tx_dt.ORTaxlot.isin(ndf.ORTaxlot.unique())]

In [165]:
# there is not any repeated taxlot ID in this taxlot feature subset
len(tx_dt_s.ORTaxlot.unique()) == tx_dt_s.shape[0]

True

In [166]:
# there is not any repeated taxlot ID in this wd data subset
len(ndf.ORTaxlot.unique()) == ndf.shape[0]

True

In [167]:
# in this case it is a one-on-one join
merged = tx_dt_s[['MapTaxlot', 'ORTaxlot', 'geometry']].merge(ndf[['wetdet_delin_number','address_location_desc', 'ORTaxlot']], 
                                                     on='ORTaxlot')

In [171]:
merged

,MapTaxlot,ORTaxlot,geometry,wetdet_delin_number,address_location_desc
0,710160000200,0407.00N10.00W1600--000000200,"MULTIPOLYGON (((443805.689 1602370.314, 443880...",WD2020-0280,W of 90266 Hwy 101
1,710160000300,0407.00N10.00W1600--000000300,"MULTIPOLYGON (((444458.870 1600237.427, 444539...",WD2020-0280,W of 90266 Hwy 101
2,81022DA01001,0408.00N10.00W22DA--000001001,"MULTIPOLYGON (((449719.082 1626698.536, 449715...",WD2020-0402,SE 2nd and E Harbor
3,81022DA01002,0408.00N10.00W22DA--000001002,"MULTIPOLYGON (((449769.062 1626696.689, 449819...",WD2020-0402,SE 2nd and E Harbor
4,81022DA01100,0408.00N10.00W22DA--000001100,"MULTIPOLYGON (((449669.104 1626700.383, 449665...",WD2020-0402,SE 2nd and E Harbor
5,810260000500,0408.00N10.00W2600--000000500,"MULTIPOLYGON (((452569.653 1623833.664, 452590...",WD2020-0468,(no discrete address) SE 12th Pl
